In [ ]:
from datasets import load_dataset
data = load_dataset("rotten_tomatoes")
data

In [ ]:
data["train"][0, -1]

In [ ]:
from transformers import pipeline
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device="cuda:0"
)